# AI as Judge

[G-Eval](https://deepeval.com/docs/metrics-llm-evals) is a framework that uses LLM as a judge to evaluate LLM outputs. The evaluation can be based on any criteria. G-Eval is implemented by a library called [DeepEval](https://deepeval.com/) which includes a broader set of tests.


In [1]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [2]:
from openai import OpenAI
import os

document_folder = "../../05_src/documents/"
blue_cross_file = "the_blue_cross.txt"
file_path = os.path.join(document_folder, blue_cross_file)

with open(file_path, "r", encoding="utf-8") as f:
    blue_cross_text = f.read()

In [3]:
blue_cross_text


'The Blue Cross\nBy G. K. Chesterton \nFrom the book "The innocence of Father Brown"\n\nBetween the silver ribbon of morning and the green glittering ribbon ofsea, the boat touched Harwich and let loose a swarm of folk like flies,among whom the man we must follow was by no means conspicuous--norwished to be. There was nothing notable about him, except a slightcontrast between the holiday gaiety of his clothes and the officialgravity of his face. His clothes included a slight, pale grey jacket,a white waistcoat, and a silver straw hat with a grey-blue ribbon. Hislean face was dark by contrast, and ended in a curt black beard thatlooked Spanish and suggested an Elizabethan ruff. He was smoking acigarette with the seriousness of an idler. There was nothing about himto indicate the fact that the grey jacket covered a loaded revolver,that the white waistcoat covered a police card, or that the straw hatcovered one of the most powerful intellects in Europe. For this wasValentin himself, the h

In [4]:
instructions = "You are an helpful assistant that summarizes works of fiction with a quirky and bubbly approach."
PROMPT = """
    Summarize the following story in at most four paragraphs. Please include all key characters and plot points.
    <story>
    {story}
    </story>
    In addition to the summary, add an introduction paragraph where you greet the reader and a conclusion where you share an opinion about the story.
"""

In [6]:
PROMPT.format(story="whatever I want")

'\n    Summarize the following story in at most four paragraphs. Please include all key characters and plot points.\n    <story>\n    whatever I want\n    </story>\n    In addition to the summary, add an introduction paragraph where you greet the reader and a conclusion where you share an opinion about the story.\n'

In [7]:
import os
client = OpenAI(base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1', 
                api_key='any value',
                default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')})
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=instructions,
    input=[
        {"role": "user", 
         "content": PROMPT.format(story=blue_cross_text)}
    ],
    temperature=0.7
)

In [8]:
response.output_text

'Hello, fabulous reader! 🌟 Buckle up for a delightful romp through a tale of cleverness and crime in G.K. Chesterton\'s "The Blue Cross," where characters are as quirky as a cat in a tutu! This story features the sharp-witted detective Aristide Valentin and the enigmatic Father Brown, whose interactions lead to a clever twist. Let\'s dive into this whimsical world of intrigue!\n\nValentin, the head of the Paris police, arrives in London on a mission to capture the notorious criminal Flambeau, who\'s known for his grand robberies and clever disguises. As he navigates the bustling streets, Valentin encounters Father Brown, a rather unassuming priest who seems oblivious to the chaos around him. Our detective soon realizes that Flambeau is likely disguising himself among the crowd during the Eucharistic Congress. But, oh, the challenge! Flambeau’s towering height is his only flaw, making him hard to miss—if only Valentin could find him!\n\nAs Valentin pursues his lead, he discovers peculia

# Answer Relevancy

The answer relevancy metric evaluates how relevant the actual output of the LLM app is compared to the provided input. This metric is self-explaining in the sense that the output includes a reason for the metric score.

The metric is calculated as:

$$
AnswerRelevancy=\frac{NumberRelevantStatements}{TotalStatements}
$$

Reference: [Answer Relevancy](https://deepeval.com/docs/metrics-answer-relevancy). 

In [9]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval.models import GPTModel

model = GPTModel(
    model="gpt-4o-mini",
    temperature=0,
    # api_key='any value',
    default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')},
    base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1',
)

metric = AnswerRelevancyMetric(
    threshold=0.95,
    include_reason=True,
    model=model,
    
)

test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text,
    
)

In [10]:
metric.measure(test_case)

Output()

1.0

In [11]:
from IPython.display import display, Markdown
display(Markdown(f'**Score**: {metric.score}'))
display(Markdown(f'**Reason**: {metric.reason}'))

**Score**: 1.0

**Reason**: The score is 1.00 because the response was entirely relevant, providing a clear summary of the story while including all key characters and plot points as requested. There were no irrelevant statements, making the summary effective and focused.

In [15]:
metric.reason

'The score is 1.00 because the response was entirely relevant, providing a clear summary of the story while including all key characters and plot points as requested. There were no irrelevant statements, making the summary effective and focused.'

In [16]:
metric.score, metric.reason

(1.0,
 'The score is 1.00 because the response was entirely relevant, providing a clear summary of the story while including all key characters and plot points as requested. There were no irrelevant statements, making the summary effective and focused.')

# Other Metrics

Other useful metric functions include:

+ [Faithfulness](https://deepeval.com/docs/metrics-faithfulness): evaluates whether the `actual_output` factually aligns with the contents of  `retrieval_context`. 
+ [Contextual Precision](https://deepeval.com/docs/metrics-contextual-precision): evaluates whether nodes in your `retrieval_context` that are relevant to the given input are ranked higher than irrelevant ones. 
+ [Contextual Recall](https://deepeval.com/docs/metrics-contextual-recall): evaluates the extent of which the retrieval_context aligns with the expected_output. 
+ [Contextual Relevancy](https://deepeval.com/docs/metrics-contextual-relevancy): evaluates the overall relevance of the information presented in your retrieval_context for a given input. 

# G-Eval

[G-Eval](https://deepeval.com/docs/metrics-llm-evals) is a framework that uses LLM-as-a-judge with chain-of-thoughts (CoT) to evaluate LLM outputs based on ANY custom criteria. The G-Eval metric is the most versatile type of metric deepeval offers.

In [17]:
instructions = "You are an helpful assistant that specializes in works of fiction."
PROMPT = """
    Based on the story below, answer the question provided.
    <story>
    {story}
    </story>
    <question>
    Who is the main antagonist in the story and what motivates their actions?
    </question>
"""

In [18]:
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=instructions,
    input=[
        {"role": "user", 
         "content": PROMPT.format(story=blue_cross_text)}
    ],
    temperature=0.7
)

In [19]:
response.output_text

'The main antagonist in the story is Flambeau, a notorious criminal known for his elaborate and ingenious thefts. His motivation for his actions is primarily driven by the thrill of committing crimes, particularly through clever and theatrical means. Flambeau enjoys outsmarting authorities, such as the police, and relishes the challenge of evading capture. In this specific narrative, he is attempting to steal a valuable silver cross with sapphires, which he has learned about through his connections and observations. His character embodies a mix of flamboyance and cunning, making him a formidable adversary for the protagonist, Father Brown.'

## Evaluation Criteria

The most straightforward way to establish a metric is by using a single criteria.

In [20]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the context.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

In [21]:
test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)
evaluate(test_cases=[test_case], metrics=[correctness_metric])

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ✅ Correctness [GEval] (score: 0.9319241694623616, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The response accurately identifies Flambeau as the main antagonist and provides a clear explanation of his motivations, including his thrill for committing crimes and outsmarting authorities. It effectively references specific details from the story, such as his attempt to steal the valuable silver cross, which aligns well with the context provided in the input. The only minor shortcoming is the lack of mention of Flambeau's past as a notorious criminal, which could have further strengthened the response., error: None)

For test case:

  - input: 
    Based on the story below, answer the question provided.
    <story>
    The Blue Cross
By G. K. Chesterton 
From the book "The innocence of Father Brown"

Between the silver ribbon of morning and the green glittering ribbon ofsea, the boat touched Harwich and let loose a swarm of folk like flies,am

✓ Evaluation completed 🎉! (time taken: 7.02s | token cost: 0.0016344 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Correctness [GEval]', threshold=0.5, success=True, score=0.9319241694623616, reason="The response accurately identifies Flambeau as the main antagonist and provides a clear explanation of his motivations, including his thrill for committing crimes and outsmarting authorities. It effectively references specific details from the story, such as his attempt to steal the valuable silver cross, which aligns well with the context provided in the input. The only minor shortcoming is the lack of mention of Flambeau's past as a notorious criminal, which could have further strengthened the response.", strict_mode=False, evaluation_model='gpt-4o-mini', error=None, evaluation_cost=0.0016344, verbose_logs='Criteria:\nDetermine whether the actual output is factually correct based on the context. \n \nEvaluation Steps:\n[\n    "1. Identify the context provided in the input to establish the factua

## Evaluation Steps 

G-Eval is flexible in many ways: notice that we can establish an evaluation criteria or a set of evaluation steps, that can help in guiding the model to follow specific steps to perform the evaluation.

In [22]:
...

correctness_metric = GEval(
    name="Correctness",
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'input'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are not OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

In [23]:
test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)
result = evaluate(test_cases=[test_case], metrics=[correctness_metric])

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ✅ Correctness [GEval] (score: 0.8474044753686112, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The response accurately identifies Flambeau as the main antagonist and provides a clear motivation for his actions, emphasizing his thrill-seeking nature and cunning. However, it could benefit from more detail regarding his specific methods and the context of his criminal activities, which were elaborately described in the story. The omission of these details slightly detracts from the completeness of the answer., error: None)

For test case:

  - input: 
    Based on the story below, answer the question provided.
    <story>
    The Blue Cross
By G. K. Chesterton 
From the book "The innocence of Father Brown"

Between the silver ribbon of morning and the green glittering ribbon ofsea, the boat touched Harwich and let loose a swarm of folk like flies,among whom the man we must follow was by no means conspicuous--norwished to be. There was nothin

✓ Evaluation completed 🎉! (time taken: 3.11s | token cost: 0.00153705 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [24]:
result.model_dump()

{'test_results': [{'name': 'test_case_0',
   'success': True,
   'metrics_data': [{'name': 'Correctness [GEval]',
     'threshold': 0.5,
     'success': True,
     'score': 0.8474044753686112,
     'reason': 'The response accurately identifies Flambeau as the main antagonist and provides a clear motivation for his actions, emphasizing his thrill-seeking nature and cunning. However, it could benefit from more detail regarding his specific methods and the context of his criminal activities, which were elaborately described in the story. The omission of these details slightly detracts from the completeness of the answer.',
     'strict_mode': False,
     'evaluation_model': 'gpt-4o-mini',
     'error': None,
     'evaluation_cost': 0.00153705,
     'verbose_logs': 'Criteria:\nNone \n \nEvaluation Steps:\n[\n    "Check whether the facts in \'actual output\' contradicts any facts in \'input\'",\n    "You should also heavily penalize omission of detail",\n    "Vague language, or contradictin